In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q hazm
!pip install -q parsivar
!pip install -q clean-text[gpl]

ERROR: parsivar 0.2.3 has requirement nltk==3.4.5, but you'll have nltk 3.3 which is incompatible.
ERROR: hazm 0.7.0 has requirement nltk==3.3, but you'll have nltk 3.4.5 which is incompatible.


In [ ]:
from hazm import *
import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import fileinput

import argparse
import sys
import re

import emoji
import string

In [ ]:
data = pd.read_excel(r'/content/drive/MyDrive/Karimi/Data/negative.xlsx')
data

,review,sentiment
0,@bisetoun @rstglili ببینا😡اول صبح غر نزن کسکشِ...,-1.0
1,توی مملکتی که حق رو دارند به این طبقه ویژه! می...,-1.0
2,لعنة اللّه علیهم 😡🤬😠 گمم نکنید😡 @Lillmeiow @Im...,-1.0
3,#روانی بمیر دیگه کسکش مگه ما مسخره توییم😡 @had...,-1.0
4,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,-1.0
...,...,...
13802,پ.ن :نسرین جون دقیقا از کدوم پیام رسان استفاده...,NaN
13803,عه😠😡 @dandunnazan اگه؟ کسکش تو زنمی 😡 @srmahnd...,NaN
13804,مجبور شدن بیان‌ طرح #مسکن_ملی راه بندازن (اسمش...,NaN
13805,بعد ۴سال مجبور شدن دوباره صادر کنن با کلی هزین...,NaN


In [ ]:
lemmatizer = Lemmatizer()

In [ ]:
def remove_space(text):
    translation_src = "\u200b\u200d_-,.\n"
    translation_dst = "\u200c\u200f     "
    text = re.sub(translation_src, ' ', text)
    text = re.sub(translation_dst, ' ', text)
    text = re.sub('\u200f', ' ', text)
    return text

In [ ]:
def remove_punct(text):
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[۱-۹]+', '', text)
    return text

In [ ]:
def emoji(text): 
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
def remove_shadda(text):
     text = re.sub('\u0651', '', text) #ARABIC SHADDA
     text = re.sub('\u064a', '', text) #ARABIC LETTER YEH  
     text = re.sub('\u0649', '', text) #ARABIC LETTER ALEF MAKSURA  
     text = re.sub('\u0652', '', text) #ARABIC SUKUN 
     text = re.sub('\u064b', '', text) #ARABIC FATHATAN 
     text = re.sub('\u064e', '', text) #ARABIC FATHA 
     text = re.sub('\u0650', '', text) #ARABIC KASRA 
     text = re.sub('\xa0',  ' ', text) #BAD SPACES 
     return text

In [ ]:
data['review'] = data['review'].apply(lambda x: remove_shadda(x))

In [ ]:
data['review'] = data['review'].apply(lambda x: remove_punct(x))
data['review'] = data['review'].str.replace('[{}]'.format(string.punctuation), '')
clean = re.compile('<.*?>')

In [ ]:
regex = re.compile('[a-zA-Z]')
data['review'] = data['review'].map(lambda x: re.sub('[a-zA-Z]', '', x))
data['review'] = data['review'].map(lambda x: re.sub(r'&',' ', x))
data['review'] = data['review'].map(lambda x: re.sub('<[^>]+>', '', x))
data['review'] = data['review'].map(lambda x: re.sub(' href\s*=\s*\"[^\"]*', '', x))
data['review'] = data['review'].map(lambda x: re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', x)) # remove URLs
data['review'] = data['review'].map(lambda x: re.sub('@[^\s]+', 'AT_USER', x))
data['review'] = data['review'].map(lambda x: re.sub(r'#([^\s]+)', r'\1', x))
data['review'] = data['review'].apply(lambda x: remove_punct(x))
data['review'] = data['review'].str.replace('[{}]'.format(string.punctuation), '')
clean = re.compile('<.*?>')
data['review'] = data['review'].map(lambda x: re.sub(clean,'', x))
data['review'] = data['review'].map(lambda x: re.sub('\u200c',' ', x))  

In [ ]:
data['review'] = data['review'].apply(lambda x: remove_space(x))

In [ ]:
data['review'] = data['review'].apply(lambda x: emoji(x))

In [ ]:
data['review']

0          ببینااول صبح غر نزن کسکش سانسورچی صبح اول صب...
1        توی مملکتی که حق رو دارند به این طبقه ویژه مید...
2        لعنة الله علیهم  گمم نکنید    انقدر عشق منو ای...
3        روانی بمیر دیگه کسکش مگه ما مسخره توییم  خب بچ...
4        در ضمن این حاج محمود کریمی که ما میشناسیم به ه...
                               ...                        
13802    پن نسرین جون دقیقا از کدوم پیام رسان استفاده م...
13803    عه  اگه؟ کسکش تو زنمی   فدا سرتواللابا این طیا...
13804    مجبور شدن بیان  طرح مسکنملی راه بندازن اسمشو ع...
13805    بعد سال مجبور شدن دوباره صادر کنن با کلی هزینه...
13806    اگه میخوای بکشی شون ،چرا زجر کش میکنی لامصب ؟    
Name: review, Length: 13807, dtype: object

In [ ]:
nan_value = float("NaN")
data.replace(" ", nan_value, inplace=True)
data.dropna(subset = ["review"], inplace=True)

In [ ]:
data['review']

0          ببینااول صبح غر نزن کسکش سانسورچی صبح اول صب...
1        توی مملکتی که حق رو دارند به این طبقه ویژه مید...
2        لعنة الله علیهم  گمم نکنید    انقدر عشق منو ای...
3        روانی بمیر دیگه کسکش مگه ما مسخره توییم  خب بچ...
4        در ضمن این حاج محمود کریمی که ما میشناسیم به ه...
                               ...                        
13802    پن نسرین جون دقیقا از کدوم پیام رسان استفاده م...
13803    عه  اگه؟ کسکش تو زنمی   فدا سرتواللابا این طیا...
13804    مجبور شدن بیان  طرح مسکنملی راه بندازن اسمشو ع...
13805    بعد سال مجبور شدن دوباره صادر کنن با کلی هزینه...
13806    اگه میخوای بکشی شون ،چرا زجر کش میکنی لامصب ؟    
Name: review, Length: 13804, dtype: object

In [ ]:
data = data.drop_duplicates(subset=['review'], keep='first')
data = data.reset_index(drop=True)
data

,review,sentiment
0,ببینااول صبح غر نزن کسکش سانسورچی صبح اول صب...,-1.0
1,توی مملکتی که حق رو دارند به این طبقه ویژه مید...,-1.0
2,لعنة الله علیهم گمم نکنید انقدر عشق منو ای...,-1.0
3,روانی بمیر دیگه کسکش مگه ما مسخره توییم خب بچ...,-1.0
4,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,-1.0
...,...,...
11074,پن نسرین جون دقیقا از کدوم پیام رسان استفاده م...,NaN
11075,عه اگه؟ کسکش تو زنمی فدا سرتواللابا این طیا...,NaN
11076,مجبور شدن بیان طرح مسکنملی راه بندازن اسمشو ع...,NaN
11077,بعد سال مجبور شدن دوباره صادر کنن با کلی هزینه...,NaN


In [ ]:
data['comment_len_by_words'] = data['review'].apply(lambda t: len(hazm.word_tokenize(t)))

In [ ]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 0 	Max: 2663


In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(data, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 73.26% of the whole!


In [ ]:
minlim, maxlim = 3, 2110

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
   
    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
# cleaning comments
data['cleaned_comment'] = data['review'].apply(cleaning)


# calculate the length of comments based on their words
data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)
data = data.drop_duplicates(subset=['cleaned_comment'], keep='first')

data.head()

,review,sentiment,comment_len_by_words,cleaned_comment,cleaned_comment_len_by_words
0,ببینااول صبح غر نزن کسکش سانسورچی صبح اول صب...,-1.0,26,ببینااول صبح غر نزن کسکش سانسورچی صبح اول صبح،...,25
1,توی مملکتی که حق رو دارند به این طبقه ویژه مید...,-1.0,324,توی مملکتی که حق رو دارند به این طبقه ویژه مید...,319
2,لعنة الله علیهم گمم نکنید انقدر عشق منو ای...,-1.0,112,لعنة الله علیهم گمم نکنید انقدر عشق منو اینور ...,111
3,روانی بمیر دیگه کسکش مگه ما مسخره توییم خب بچ...,-1.0,47,روانی بمیر دیگه کسکش مگه ما مسخره توییم خب بچه...,47
4,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,-1.0,138,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,137


In [ ]:
data = data[['cleaned_comment', 'sentiment']]
data.columns = ['review', 'sentiment']
data

,review,sentiment
0,ببینااول صبح غر نزن کسکش سانسورچی صبح اول صبح،...,-1.0
1,توی مملکتی که حق رو دارند به این طبقه ویژه مید...,-1.0
2,لعنة الله علیهم گمم نکنید انقدر عشق منو اینور ...,-1.0
3,روانی بمیر دیگه کسکش مگه ما مسخره توییم خب بچه...,-1.0
4,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,-1.0
...,...,...
11074,پن نسرین جون دقیقا از کدوم پیام رسان استفاده م...,NaN
11075,عه اگه؟ کسکش تو زنمی فدا سرتواللابا این طیاره ...,NaN
11076,مجبور شدن بیان طرح مسکنملی راه بندازن اسمشو عو...,NaN
11077,بعد سال مجبور شدن دوباره صادر کنن با کلی هزینه...,NaN


In [ ]:
def clean(text):
    intab='۱۲۳۴۵۶۷۸۹۰١٢٣٤٥٦٧٨٩٠'
    outtab='12345678901234567890'
    translation_table = str.maketrans(intab, outtab)
    text = text.translate(translation_table)
    tex = re.sub(r"(?:\@|https?\://)\s+"," ",text)
    text = re.findall(r"[A-Za-z._]+|[^A-Za-z\W]+",text,re.UNICODE)
    text = ' '.join(word for word in text)
    cleanr = re.compile('<.*?>')
    cleanr = re.compile(r'<[^>]+>')
    text = re.sub(cleanr,'',text)
    text = re.sub(r"""\d""",'',text)
    text = re.sub('\r?\n','.',text)
    text = re.sub(r"-{3}",'',text)
    text = re.sub(r"-{2}",'',text)
    text = re.sub(r"""\s*\.{3,}""",u'.',text)
    text = re.sub(r"""\s*\.{2,}""",u'.',text)
    text = re.sub(r"""\s+(ن؟می)\s+""",r'\1',text)
    text = re.sub(r"""(!){2,}""",r'\1',text)
    text = re.sub(r"""(/ ){2,}""",'',text)
    text = re.sub(r"""( /){2,}""",'',text)
    text = re.sub(r"""(//){2,}""",'',text)
    text = re.sub(r"""(/){2,}""",'',text)
    text = re.sub(r"""(؟){2,}""",r'\1',text)
    text = re.sub(r"""_+""","",text)
    text = re.sub(r"""[ ]+""",r' ',text)
    text = re.sub(r"""([\n]+)[\t]*""",r'\1',text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    p = re.compile(r'<.*?>')
    #text = re.findall(r"[^A-Za-z\W]+",text,re.UNICODE)
    #text = re.sub(r"""(/){1,}""",'-',text)
    #text = re.findall(r"[^\dA-Za-z\W]+|\d+",text,re.UNICODE)
    #text = ' '.join(word for word in text)
    #text = p.sub('', text)
    #text = re.sub(r"""\\d{2-3}\s+[^\dA-Za-z\W]\s+\\d{2-3}""",r'\1',text)
    #text = re.sub(r"""\s+(ی)ها|?(?(ن))""",r'\1',text)
    text = re.sub(r"""product""","",text)
    text = re.sub(r"""dkp""","",text)
    text = re.sub(r"""br""","",text)
    text = re.sub(r"""mm""","",text)
    text = text.strip()

    return text

In [ ]:
data['review'] = data['review'].apply(lambda x: clean(x))

In [ ]:
data['sentiment'] = data['sentiment'].fillna(-1)

In [ ]:
data

,review,sentiment
0,ببینااول صبح غر نزن کسکش سانسورچی صبح اول صبح ...,-1.0
1,توی مملکتی که حق رو دارند به این طبقه ویژه مید...,-1.0
2,لعنة الله علیهم گمم نکنید انقدر عشق منو اینور ...,-1.0
3,روانی بمیر دیگه کسکش مگه ما مسخره توییم خب بچه...,-1.0
4,در ضمن این حاج محمود کریمی که ما میشناسیم به ه...,-1.0
...,...,...
11074,پن نسرین جون دقیقا از کدوم پیام رسان استفاده م...,-1.0
11075,عه اگه کسکش تو زنمی فدا سرتواللابا این طیاره ه...,-1.0
11076,مجبور شدن بیان طرح مسکنملی راه بندازن اسمشو عو...,-1.0
11077,بعد سال مجبور شدن دوباره صادر کنن با کلی هزینه...,-1.0


In [ ]:
data.to_excel(r'/content/drive/MyDrive/Karimi/Data/new_negative.xlsx', index = False)